In [6]:

import numpy as np
import pandas as pd
import operator


cut_programs = np.load('../data/cut_Programs.npy')
cut_Question = np.load('../data/cut_Questions.npy')

def add_word_dict(w, word_dict):
    if not w in word_dict:
        word_dict[w] = 1
    else:
        word_dict[w] += 1

def RmOutOfVoc(cut_programs, cut_Question):
    word_dict = dict()
    for program in cut_programs:
        for lines in program:
            for line in lines:
                for w in line:
                    add_word_dict(w, word_dict)

    for question in cut_Question:
        lines = question[0]
        for line in lines:
            for w in line:
                add_word_dict(w, word_dict)

        for i in range(1, 7):
            line = question[i]
            for w in line:
                add_word_dict(w, word_dict)
                
    word_dict = sorted(word_dict.items(), key=operator.itemgetter(1), reverse=True)
    VOC_SIZE = 15000
    VOC_START = 25

    voc_dict = word_dict[VOC_START:VOC_START+VOC_SIZE]
    return voc_dict

voc_dict = RmOutOfVoc(cut_programs, cut_Question)
np.save('voc_dict.npy', voc_dict)

print("done")

done


In [7]:
import numpy as np
import pandas as pd

cut_programs = np.load('../data/cut_Programs.npy')                                                                
voc_dict = np.load('./voc_dict.npy')                                                                              
                                                                                                                  
sample_doc = []                                                                                                   
for cut_program in cut_programs:                                                                                  
    for episode in cut_program:                                                                                   
        for line in episode:                                                                                      
            sample_line = ''                                                                                      
            for w in line:                                                                                        
                if w in voc_dict:                                                                                 
                    sample_line += w+' '                                                                          
            sample_doc.append(sample_line)                                                                        
                                                                                                                  
np.save('./sample_doc.npy', sample_doc)    
print("done")

done


In [ ]:
"""
TODO:
1.將整個sample_doc變成word2vec model
2.利用model產生training data     e.g. 隨機選2句，相似度高於0.7(?)的直接給1，其他0
3.剩下的照舊
"""

In [ ]:
import random
import numpy as np
import pandas as pd

sample_doc = np.load('./sample_doc.npy')

with open('word2vec_sentence.txt', 'w') as file:
    for sentence in sample_doc:
        line = ""
        for word in sentence:
            line += word + ' '
        file.write(line + '\n')
    file.close()

print("done")

In [11]:
from gensim.models import word2vec
from gensim import models
sentences = word2vec.Text8Corpus("word2vec_sentence.txt")
word2vec_model = word2vec.Word2Vec(sentences)

#保存模型，供日後使用
word2vec_model.save("word2vec_model.bin")

#模型讀取方式
# word2vec_model = word2vec.Word2Vec.load("word2vec_model.bin")
print("done")

done


In [34]:
from gensim.models import word2vec
from gensim import models
from sklearn.feature_extraction.text import HashingVectorizer
import numpy as np
import pandas as pd
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import roc_auc_score
import csv

sample_doc = np.load('./sample_doc.npy')
word2vec_model = word2vec.Word2Vec.load("word2vec_model.bin")

NUM_TRAIN = 50000
train_valid = 0.3
len = sample_doc.shape[0]
def generate_training_data():
    Xs, Ys = [], []
    replaceList = [" ", '\u3000', '\t',]
    for i in range(NUM_TRAIN):
        line_id = random.randint(0, len-2)
        if sample_doc[line_id] != "" and sample_doc[line_id + 1] != "":
            Xs.append([sample_doc[line_id], 
                      sample_doc[line_id + 1]])
            a = sample_doc[line_id]
            b = sample_doc[line_id + 1]
            for ch in replaceList:
                if ch in a:
                    a = a.replace(ch, "")
                if ch in b:
                    b = b.replace(ch, "")
            if a and b:#if a and b are not empty
                similarity = word2vec_model.n_similarity(a, b)
                if similarity > train_valid:
                    Ys.append(1)
                else:
                    Ys.append(0)

    return Xs, Ys


Xs, Ys = generate_training_data()

X_vec = []
Y_vec = []
for X, Y in zip(Xs, Ys):
    sentence = X[0] + X[1]
    if sentence == "":
        print("empty")
        continue
    X_vec.append(sentence)
    Y_vec.append(Y)
Xs = X_vec
Ys = Y_vec

with open('random_word2vec.csv', 'w', newline='') as csvfile:
    fieldnames = ['sentence', 'relation']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    

for x, y in zip(Xs, Ys):
    with open('random_word2vec.csv', 'a', newline='') as csvfile:
        fieldnames = ['sentence', 'relation']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writerow({'sentence': x, 'relation': y})

print("done")

done


In [40]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import roc_auc_score
import pandas as pd

def get_stream(path, size):
    for chunk in pd.read_csv(path, chunksize=size):
        yield chunk



hashvec = HashingVectorizer(n_features=2**10)
# loss='log' gives logistic regression
clf = SGDClassifier(loss='log', n_iter=100)

num_lines = sum(1 for line in open('random_word2vec.csv'))
batch_size = num_lines//20
stream = get_stream(path='random_word2vec.csv', size=batch_size)


classes = np.array([0, 1])
train_auc, val_auc = [], []

# we use one batch for training and another for validation in each iteration
NUM_TRAIN = num_lines
iters = int((NUM_TRAIN+batch_size-1)/(batch_size*2))

for i in range(iters):
    try:
        batch = next(stream)
        X_train, y_train = batch['sentence'], batch['relation']
        if X_train is None:
            break

        X_train = hashvec.transform(X_train)
        clf.partial_fit(X_train, y_train, classes=classes)
        train_auc.append(roc_auc_score(y_train, clf.predict_proba(X_train)[:,1]))
    
        # validate
        batch = next(stream)
        X_val, y_val = batch['sentence'], batch['relation']
        score = roc_auc_score(y_val, clf.predict_proba(hashvec.transform(X_val))[:,1])
        val_auc.append(score)
        print('[%d/%d] %f' % ((i+1)*(batch_size*2), NUM_TRAIN, score))
    except Exception as e:
        print(e)

print("done")

/home/jrchang/workspace/anaconda/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:73: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


[4298/42995] 0.542650
[8596/42995] 0.530756
[12894/42995] 0.564614
[17192/42995] 0.557949
[21490/42995] 0.553412
[25788/42995] 0.537551
[30086/42995] 0.521183
[34384/42995] 0.505890
[38682/42995] 0.515512
[42980/42995] 0.497764
done
